In [1]:
import os

In [2]:
%pwd

'/home/syorgun/CYBki--End-to-End-Chest-Cancer-Classification-using-MLflow-DVC/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/syorgun/CYBki--End-to-End-Chest-Cancer-Classification-using-MLflow-DVC'

In [5]:
%pip install dagshub

Note: you may need to restart the kernel to use updated packages.


In [6]:
import dagshub
dagshub.init(repo_owner='CYBki', repo_name='CYBki--End-to-End-Chest-Cancer-Classification-using-MLflow-DVC', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as CYBki

Initialized MLflow to track repo "CYBki/CYBki--End-to-End-Chest-Cancer-Classification-using-MLflow-DVC"

Repository CYBki/CYBki--End-to-End-Chest-Cancer-Classification-using-MLflow-DVC initialized!

In [7]:
import tensorflow as tf
from tensorflow import keras

keras.__version__ = tf.__version__


2025-10-25 13:48:35.712289: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-25 13:48:35.713935: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-25 13:48:35.743198: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-25 13:48:35.743863: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-25 13:48:36.204927: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [8]:
model = tf.keras.models.load_model('artifacts/training/trained_model.h5')

2025-10-25 13:48:38.223480: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-10-25 13:48:38.223503: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: syorgun-EXCALIBUR-G870
2025-10-25 13:48:38.223507: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: syorgun-EXCALIBUR-G870
2025-10-25 13:48:38.223744: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 550.163.1
2025-10-25 13:48:38.223757: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 550.163.1
2025-10-25 13:48:38.223760: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 550.163.1


In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path 
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/trained_model.h5"),
            training_data=Path("artifacts/data_ingestion/Chest-CT-Scan-data"),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(
                    self.model, 
                    "model", 
                    registered_model_name="VGG16Model"
                )
            else:
                mlflow.keras.log_model(self.model, "model")


In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e



[2025-10-25 13:48:38,562: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-10-25 13:48:38,564: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-25 13:48:38,565: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 5s 717ms/step - loss: 10.5532 - accuracy: 0.5686
[2025-10-25 13:48:44,104: INFO: common: json file saved at: scores.json]


2025/10/25 13:48:45 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmpssak2qrh/model/data/model/assets
[2025-10-25 13:48:45,707: INFO: builder_impl: Assets written to: /tmp/tmpssak2qrh/model/data/model/assets]


AttributeError: module 'tensorflow.keras' has no attribute '__version__'